In [1]:
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import re

In [2]:
url = 'https://www.kabum.com.br/computadores/notebooks'

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}

site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
qtd_itens = soup.find('div', id="listingCount").get_text().strip()

index = qtd_itens.find(' ')
qtd = qtd_itens[:index]

In [48]:
dic_produtos = {'marca': [], 'preço': [], 'link': []}

for i in range(1,  150):
    url_pag = f'https://www.kabum.com.br/computadores/notebooks?page_number={i}&page_size=20&facet_filters=&sort=most_searched'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, 'lxml')
    produtos = soup.find_all('div', class_=re.compile('productCard'))

    for produto in produtos:
        marca = produto.find('span', class_=re.compile('nameCard')).get_text().strip()
        preço = produto.find('span', class_=re.compile('priceCard')).get_text().strip()
        link = produto.find('a', attrs={'href': re.compile('^')}).get('href')

        #print(marca, preço)

        assert isinstance(marca, object)
        dic_produtos['marca'].append(marca)
        dic_produtos['preço'].append(preço)
        dic_produtos['link'].append('https://www.kabum.com.br/' + link)

    #print(url_pag)
    
#data = pd.read_csv("preço_notebooks.csv")
# nome_arquivo_excel = 'preço_notebooks_excel.xlsx'
# data.to_excel(nome_arquivo_excel, index=False)
# df.to_csv("preço_notebooks.csv", index=False)

df = pd.DataFrame(dic_produtos)
df.head()

,marca,preço,link
0,Notebook Gamer Lenovo Gaming 3i Intel Core i5-...,"R$ 4.199,99",https://www.kabum.com.br//produto/373931/noteb...
1,"Notebook Asus AMD Ryzen 5-3500U, 8GB RAM, SSD ...","R$ 3.099,99",https://www.kabum.com.br//produto/334039/noteb...
2,"Notebook Acer Aspire 5 Ryzen 7-5700U, 8GB RAM,...","R$ 3.599,99",https://www.kabum.com.br//produto/308368/noteb...
3,Notebook Gamer Acer Nitro 5 Intel Core i7-1180...,"R$ 5.099,99",https://www.kabum.com.br//produto/382898/noteb...
4,"Notebook Asus Intel Core i3 1005G1, 4GB RAM, 2...","R$ 1.949,99",https://www.kabum.com.br//produto/369923/noteb...


In [47]:
df.tail(5)

,marca,preço,link
3975,"Notebook Dell Gamer G15 - Fhd 120hz, Ryzen 5 5...",R$ ---,https://www.kabum.com.br//produto/323894/noteb...
3976,"Notebook Lenovo S145, Tela 15.6"", Ryzen 5, 8GB...",R$ ---,https://www.kabum.com.br//produto/201220/noteb...
3977,"Fonte Knup 19.5V, Para Notebook Dell 4.62A - K...",R$ ---,https://www.kabum.com.br//produto/293816/fonte...
3978,Teclado P/ Notebook Lenovo Thinkpad T460s T470s,R$ ---,https://www.kabum.com.br//produto/290971/tecla...
3979,Bateria Notebook - Lg R580 - Preta,R$ ---,https://www.kabum.com.br//produto/224793/bater...


In [45]:
df['marca'] = df['marca'].str.upper()

produto = input('Qual produto você deseja? ')
produto = produto.upper()

resultado = df[df['marca'].str.contains(fr'{produto}')]
resultado

Qual produto você deseja? teclado


,marca,preço,link
297,"COMPUTADOR COMPLETO FÁCIL AMD RYZEN 5 5600G, V...","R$ 3.089,00",https://www.kabum.com.br//produto/379033/compu...
298,"COMPUTADOR COMPLETO FÁCIL AMD RYZEN 5 5600G, V...","R$ 3.289,00",https://www.kabum.com.br//produto/379062/compu...
326,TECLADO P/ NOTEBOOK DELL LATITUDE E5420 PADR...,"R$ 169,99",https://www.kabum.com.br//produto/347051/tecla...
349,TECLADO NOTEBOOK FTI PARA LENOVO THINKPAD T490...,"R$ 309,99",https://www.kabum.com.br//produto/352823/tecla...
372,TECLADO P/ NOTEBOOK LENOVO THINKPAD E470 - B...,"R$ 349,00",https://www.kabum.com.br//produto/297032/tecla...
...,...,...,...
5895,TECLADO NOTEBOOK - CÓDIGOS 050X15 FRAME KAZUK,R$ ---,https://www.kabum.com.br//produto/368192/tecla...
5897,TECLADO NOTEBOOK ACER ASPIRE E5-571PG SEM FRAM...,R$ ---,https://www.kabum.com.br//produto/351874/tecla...
5911,"TECLADO NOTEBOOK, FRAME, PRETO - CÓDIGOS NSK-L...",R$ ---,https://www.kabum.com.br//produto/368270/tecla...
5974,TECLADO NOTEBOOK - DELL VOSTRO 1500 - KAZUK,R$ ---,https://www.kabum.com.br//produto/346721/tecla...


In [40]:
from IPython.display import display, HTML

# função para criar os links clicáveis
def link(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

df['Links Clicáveis'] = df['link'].apply(criar_link)
df.drop(['link'], axis=1, inplace=True)

display(HTML(df.to_html(escape=False)))

,marca,preço,Links Clicáveis
0,"NOTEBOOK GAMER LENOVO GAMING 3I INTEL CORE I5-11300H, GEFORCE GTX 1650, 8GB RAM, SSD 512GB, 15.6 FULL HD, WINDOWS 11, PRETO - 82MG0009BR","R$ 4.199,99",https://www.kabum.com.br//produto/373931/notebook-gamer-lenovo-gaming-3i-intel-core-i5-11300h-geforce-gtx-1650-8gb-ram-ssd-512gb-15-6-full-hd-windows-11-preto-82mg0009br
1,"NOTEBOOK ASUS AMD RYZEN 5-3500U, 8GB RAM, SSD 256GB, 15,6', RADEON VEGA 8, WINDOWS 11 HOME, CINZA - M515DA-BR1213W","R$ 3.099,99",https://www.kabum.com.br//produto/334039/notebook-asus-amd-ryzen-5-3500u-8gb-ram-ssd-256gb-15-6-radeon-vega-8-windows-11-home-cinza-m515da-br1213w
2,"NOTEBOOK ACER ASPIRE 5 RYZEN 7-5700U, 8GB RAM, 256GB SSD NVME, TELA 15.6 IPS FULL HD, WINDOWS 11 HOME, PRATA - A515-45-R760","R$ 3.599,99",https://www.kabum.com.br//produto/308368/notebook-acer-aspire-5-ryzen-7-5700u-8gb-ram-256gb-ssd-nvme-tela-15-6-ips-full-hd-windows-11-home-prata-a515-45-r760
3,"NOTEBOOK GAMER ACER NITRO 5 INTEL CORE I7-11800H, GEFORCE GTX 1650, 8GB RAM, SSD 512GB, 15.6 FULL HD 144HZ IPS, WINDOWS 11, PRETO - AN515-57-740K","R$ 5.099,99",https://www.kabum.com.br//produto/382898/notebook-gamer-acer-nitro-5-intel-core-i7-11800h-geforce-gtx-1650-8gb-ram-ssd-512gb-15-6-full-hd-144hz-ips-windows-11-preto-an515-57-740k
4,"NOTEBOOK ASUS INTEL CORE I3 1005G1, 4GB RAM, 256GB SSD, TELA 15.6, ENDLESS OS, CINZA - X515JA-BR2750","R$ 1.949,99",https://www.kabum.com.br//produto/369923/notebook-asus-intel-core-i3-1005g1-4gb-ram-256gb-ssd-tela-15-6-endless-os-cinza-x515ja-br2750
5,"NOTEBOOK GAMER LENOVO GAMING 3I INTEL CORE I5-11300H, GEFORCE GTX1650, 8GB RAM, SSD 512GB, 15.6 FULL HD, LINUX, PRETO - 82MGS00200","R$ 3.899,99",https://www.kabum.com.br//produto/378467/notebook-gamer-lenovo-gaming-3i-intel-core-i5-11300h-geforce-gtx1650-8gb-ram-ssd-512gb-15-6-full-hd-linux-preto-82mgs00200
6,"NOTEBOOK ACER ASPIRE 5, INTEL CORE I5 11ª GEN, 8GB, SSD 256GB, 14 POL. FULL HD, LINUX GUTTA - A514-54-56LF","R$ 2.599,35",https://www.kabum.com.br//produto/401176/notebook-acer-aspire-5-intel-core-i5-11-gen-8gb-ssd-256gb-14-pol-full-hd-linux-gutta-a514-54-56lf
7,"NOTEBOOK SAMSUNG INTEL CORE I5-1135G7, 8GB RAM, SSD 256GB, 15.6 FULL HD, WINDOWS 11 HOME, CINZA - NP550XDA-KH2BR","R$ 2.879,10",https://www.kabum.com.br//produto/291436/notebook-samsung-intel-core-i5-1135g7-8gb-ram-ssd-256gb-15-6-full-hd-windows-11-home-cinza-np550xda-kh2br
8,"NOTEBOOK ACER ASPIRE 5, INTEL CORE I7 11ª GEN, 8GB, SSD 512GB, 15.6 POL. FULL HD, LINUX GUTTA - A515-56-740V","R$ 3.495,05",https://www.kabum.com.br//produto/401177/notebook-acer-aspire-5-intel-core-i7-11-gen-8gb-ssd-512gb-15-6-pol-full-hd-linux-gutta-a515-56-740v
9,"NOTEBOOK LENOVO ULTRAFINO IDEAPAD 3 RYZEN 7-5700U, 8GB, SSD 256GB, WINDOWS 11, 15.6, PRATA - 82MF0004BR","R$ 3.999,99",https://www.kabum.com.br//produto/254197/notebook-lenovo-ultrafino-ideapad-3-ryzen-7-5700u-8gb-ssd-256gb-windows-11-15-6-prata-82mf0004br
